In [138]:
#TODO: what if the error is 0????

import math

def gen_stumps(inputs, outputs):
    stumps = [] 
    
    #first we identify which values are possible
    for attr in range(len(inputs[0])):
        stumps.append({})
        for line in inputs:
            value = line[attr]
            stumps[attr][value] = []
    
    #now we generate the stumps
    for (attr, stump) in enumerate(stumps):
        for (category, wrong_elements) in stump.items():
            for (i, line) in enumerate(inputs):
                if category == line[attr] and outputs[i] == -1 or category != line[attr] and outputs[i] == 1:
                    wrong_elements.append(i)
                    
    #extra stump to represent when everything is considered 1 or everything is considered -1
    extra_stump = {-1: [], 1: []}
    for (i, output) in enumerate(outputs):
        extra_stump[1 if output == -1 else -1].append(i)
    
    stumps.append(extra_stump)
    
    return stumps

def boosting(inputs, outputs, num_iters):
    model = {}
    stumps = gen_stumps(inputs, outputs)
    
    weights = [1 / len(inputs)] * len(inputs)
    
    for i in range(num_iters):
        alpha, stump = boost_round(stumps, weights)
        model[stump] = model.get(stump, 0) + alpha
    
    return model

def compute_accuracy(model, inputs, outputs):
    correct = 0
    for i, input in enumerate(inputs):
        pred = predict(model, input)
        if pred == outputs[i]: correct += 1
    return correct / len(inputs)

def predict(model, input):
    constant_attr = len(input)
    result = 0
    for (attribute, value), alpha in model.items():
        attr_value = input[attribute] if attribute != constant_attr else 1
        result += alpha * (1 if  attr_value == value else -1)
    return 1 if result > 0 else -1

def boost_round(stumps, weights):
    best_error = 1000 #this is correct, since the max total error is 1
    best_stump = None
    wrongs = []
    
    for attribute, categories in enumerate(stumps):
        for (category_value, wrong_elements) in categories.items():
            error = 0
            for id in wrong_elements:
                error += weights[id]
            if error < best_error:
                best_error = error
                best_stump = (attribute, category_value)
                wrongs = wrong_elements
    
    alpha = 1/2 * math.log((1 - best_error) / best_error)
    
    sum = 0
    for i, wold in enumerate(weights):
        factor = -1 if i in wrongs else 1
        wnew = wold * math.exp(-alpha * factor)
        weights[i] = wnew
        sum += wnew
    
    for i, weight in enumerate(weights):
        weights[i] = weight / sum
        
    return (alpha, best_stump)

In [147]:
from numpy import genfromtxt

data = genfromtxt('tic-tac-toe.data.txt', delimiter=',', dtype=object)
random.shuffle(data)
inputs = data[:,:-1]
raw_outputs = data[:, -1:]
outputs = [1 if output == b'positive' else -1 for [output] in raw_outputs]

train_size = int(len(inputs) * 0.9)
train_in = inputs[:train_size]
train_out = outputs[:train_size] 
test_in = inputs[train_size:]
test_out = outputs[train_size:] 

model = boosting(train_in, train_out, 100)
acc = compute_accuracy(model, test_in, test_out)
print(acc)

0.7395833333333334


In [50]:
outputs = [1,1,1,1,1,1,1,-1,-1,-1]
inputs = [[0,1,1], [1,1,1], [1, 0, 0], [1,0,1], [1,0,1], [0,1,1], [0,1,1], 
              [0,1,0], [1,0,0], [0,0,1]]

model = boosting(inputs, outputs, 10)
print(model)
acc = compute_accuracy(model, inputs, outputs)
print(acc)

[(0.6931471805599453, (2, 1)), (0.5493061443340549, (0, 1)), (0.44365159750045147, (1, 1)), (0.4789198673935139, (3, 1)), (0.326685164116977, (0, 1)), (0.3965487794884476, (2, 1)), (0.2809449421468505, (0, 1)), (0.45256905194069447, (1, 1)), (0.30705657427613087, (0, 1)), (0.2806296440908112, (2, 1))]
0.9
